In [1]:
import vtk
from vtk.numpy_interface import dataset_adapter as dsa
from vmtk import vmtkscripts
import numpy as np

import os

In [2]:
datadir = '/Users/rick/projects/vmtk/vmtk-test-data/input/'

In [3]:
centerlinereader = vmtkscripts.vmtkSurfaceReader()
centerlinereader.InputFileName = os.path.join(datadir, 'aorta-centerline-branches.vtp')
centerlinereader.Execute()

Reading VTK XML surface file.


In [4]:
surfacereader = vmtkscripts.vmtkSurfaceReader()
surfacereader.InputFileName = os.path.join(datadir, 'aorta-surface-branch-split.vtp')
surfacereader.Execute()

Reading VTK XML surface file.


In [7]:
numSpheres = [1, 2]
reverseDirection = [0]
branches = []

for numberSpheres in numSpheres:
    for reverseIdx in reverseDirection:
                         
        sections = vmtkscripts.vmtkBranchSections()
        sections.Surface = surfacereader.Surface
        sections.Centerlines = centerlinereader.Surface
        sections.NumberOfDistanceSpheres = numberSpheres
        sections.ReverseDirection = reverseIdx
        sections.RadiusArrayName = 'MaximumInscribedSphereRadius'
        sections.GroupIdsArrayName = 'GroupIds'
        sections.CenterlineIdsArrayName = 'CenterlineIds'
        sections.TractIdsArrayName = 'TractIds'
        sections.BlankingArrayName = 'Blanking'

        sections.Execute()
        
        print(f'done with spheres: {numberSpheres}, reverse: {reverseIdx}')
        
        branches.append(sections.BranchSections)

done with spheres: 1, reverse: 0
done with spheres: 2, reverse: 0


In [16]:
wp = dsa.WrapDataObject(branches[1])

In [17]:
wp.PointData.keys()

[]

In [18]:
cellDataKeys = ['BranchSectionGroupIds',
                 'BranchSectionArea',
                 'BranchSectionMinSize',
                 'BranchSectionMaxSize',
                 'BranchSectionShape',
                 'BranchSectionClosed',
                 'BranchSectionDistanceSpheres']

In [19]:
objectCellDataKeys = wp.CellData.keys()
for cellKey in cellDataKeys:
    assert cellKey in objectCellDataKeys
    objectCellDataKeys.remove(cellKey)
    
assert len(objectCellDataKeys) == 0

In [20]:
assert len(wp.PointData.keys()) == 0

In [21]:
for key in cellDataKeys:
    print(f'{key} \n {wp.CellData.GetArray(key)}')

BranchSectionGroupIds 
 [0 0 0 2 2 2 2 2 3 3 3 3 3]
BranchSectionArea 
 [ 195.53117732  186.21267186  177.20153242   85.82336158   63.50445732
   62.22208797   60.78477703   63.08210028   99.06819265   64.12766266
   67.13289619   59.98268905   58.6038296 ]
BranchSectionMinSize 
 [ 15.25387687  14.66768231  14.6356421   10.89010723   8.86926931
   8.56866816   8.58205574   8.73891524  11.33372646   8.91067727
   8.87761983   8.53398992   8.73934951]
BranchSectionMaxSize 
 [ 17.08821628  16.22629607  16.01361226  11.69644525   9.35342472
   9.21275981   9.04795408   9.37937275  12.45697059   9.27120319
   9.83837421   9.13391134   8.86614888]
BranchSectionShape 
 [ 0.89265472  0.90394519  0.91395007  0.93106127  0.94823763  0.930087
  0.94850788  0.93171638  0.90983007  0.96111336  0.90234622  0.93431933
  0.98569848]
BranchSectionClosed 
 [1 1 1 0 1 1 1 1 0 1 1 1 1]
BranchSectionDistanceSpheres 
 [0 2 4 0 2 4 6 8 0 2 4 6 8]


In [22]:
wp.Points.shape

(1111, 3)

In [15]:
assert wp.Points.shape == (2065, 3)

### Number of Cells

In [25]:
print(f'pytest.mark.parametrize("expectedValue,paramid",[')
for idx, branch in enumerate(branches):
    print(f'    ({branch.GetNumberOfCells()}, {idx}),')
print('])')

pytest.mark.parametrize("expectedValue,paramid",[
    (24, 0),
    (13, 1),
])


### start indices

In [30]:
for idx, branch in enumerate(branches):
    numCells = branch.GetNumberOfCells()
    print(f'@pytest.mark.parametrize("expectedValue,paramid",[')
    for idc in range(numCells):
        bcx = branch.GetCell(idc)
        pointIdStart = bcx.GetPointId(0)
        print(f'    ({pointIdStart}, {idc}),')
    print('])')

@pytest.mark.parametrize("expectedValue,paramid",[
    (0, 0),
    (129, 1),
    (246, 2),
    (369, 3),
    (491, 4),
    (610, 5),
    (712, 6),
    (784, 7),
    (864, 8),
    (938, 9),
    (1012, 10),
    (1084, 11),
    (1157, 12),
    (1227, 13),
    (1294, 14),
    (1362, 15),
    (1440, 16),
    (1530, 17),
    (1604, 18),
    (1685, 19),
    (1760, 20),
    (1838, 21),
    (1916, 22),
    (1986, 23),
])
@pytest.mark.parametrize("expectedValue,paramid",[
    (0, 0),
    (129, 1),
    (252, 2),
    (371, 3),
    (443, 4),
    (517, 5),
    (589, 6),
    (659, 7),
    (727, 8),
    (805, 9),
    (879, 10),
    (954, 11),
    (1032, 12),
])


### end indices

In [31]:
for idx, branch in enumerate(branches):
    numCells = branch.GetNumberOfCells()
    print(f'@pytest.mark.parametrize("expectedValue,numberofpoints,paramid",[')
    for idc in range(numCells):
        bcx = branch.GetCell(idc)
        numberOfPoints = bcx.GetNumberOfPoints()
        pointIdEnd = bcx.GetPointId(numberOfPoints - 1)
        print(f'    ({pointIdEnd}, {numberOfPoints}, {idc}),')
    print('])')



@pytest.mark.parametrize("expectedValue,numberofpoints,paramid",[
    (128, 129, 0),
    (245, 117, 1),
    (368, 123, 2),
    (490, 122, 3),
    (609, 119, 4),
    (711, 102, 5),
    (783, 72, 6),
    (863, 80, 7),
    (937, 74, 8),
    (1011, 74, 9),
    (1083, 72, 10),
    (1156, 73, 11),
    (1226, 70, 12),
    (1293, 67, 13),
    (1361, 68, 14),
    (1439, 78, 15),
    (1529, 90, 16),
    (1603, 74, 17),
    (1684, 81, 18),
    (1759, 75, 19),
    (1837, 78, 20),
    (1915, 78, 21),
    (1985, 70, 22),
    (2064, 79, 23),
])
@pytest.mark.parametrize("expectedValue,numberofpoints,paramid",[
    (128, 129, 0),
    (251, 123, 1),
    (370, 119, 2),
    (442, 72, 3),
    (516, 74, 4),
    (588, 72, 5),
    (658, 70, 6),
    (726, 68, 7),
    (804, 78, 8),
    (878, 74, 9),
    (953, 75, 10),
    (1031, 78, 11),
    (1110, 79, 12),
])


### start and end locations

In [28]:
for idx, branch in enumerate(branches):
    numCells = branch.GetNumberOfCells()
    wp = dsa.WrapDataObject(branch)
    print(f'pytest.mark.prametrize("pointidstart,numberofpoints,expectedlocationstart,expectedlocationend,paramid",[')
    for idc in range(numCells):
        bcx = branch.GetCell(idc)
        numberOfPoints = bcx.GetNumberOfPoints()
        pointIdStart = bcx.GetPointId(0)
        pointIdEnd = bcx.GetPointId(numberOfPoints - 1)
        pointLocationStart = wp.Points[pointIdStart]
        pointLocationEnd = wp.Points[pointIdEnd]
        print(f'    ({pointIdStart}, {numberOfPoints}, np.array([{pointLocationStart[0]}, {pointLocationStart[1]}, {pointLocationStart[2]}]), np.array([{pointLocationEnd[0]}, {pointLocationEnd[1]}, {pointLocationEnd[2]}]), {idc}),')
    print('])')

pytest.mark.prametrize("pointidstart,numberofpoints,expectedlocationstart,expectedlocationend,paramid",[
    (0, 129, np.array([217.96841430664062, 173.62118530273438, 13.255617141723633]), np.array([218.3564910888672, 173.5325927734375, 13.078214645385742]), 0),
    (129, 117, np.array([220.60513305664062, 168.85765075683594, 14.689851760864258]), np.array([220.7154998779297, 168.8603515625, 14.662433624267578]), 1),
    (246, 123, np.array([220.60513305664062, 161.35403442382812, 14.91808795928955]), np.array([221.1270294189453, 161.36172485351562, 14.869494438171387]), 2),
    (369, 122, np.array([217.96841430664062, 153.799560546875, 16.24483871459961]), np.array([218.7977752685547, 153.759033203125, 15.950116157531738]), 3),
    (491, 119, np.array([220.60513305664062, 147.16163635253906, 16.457529067993164]), np.array([220.84371948242188, 147.17149353027344, 16.445602416992188]), 4),
    (610, 102, np.array([214.2646026611328, 138.3168182373047, 20.781129837036133]), np.array([21

In [25]:
for idx in range(numCells):
    print(f'For cell: {idx}')
    bcx = branches.GetCell(idx)
    numberOfPoints = bcx.GetNumberOfPoints()
    pointIdStart = bcx.GetPointId(0)
    pointIdEnd = bcx.GetPointId(numberOfPoints - 1)
    pointLocationStart = wp.Points[pointIdStart]
    pointLocationEnd = wp.Points[pointIdEnd]
    print(f'Number Of Points: {numberOfPoints} \n',
          f'Point Id Start: {pointIdStart} \n',
          f'Point Id End: {pointIdEnd} \n',
          f'np.array({pointLocationStart}), np.array({pointLocationEnd}) \n')

For cell: 0
Number Of Points: 129 
 Point Id Start: 0 
 Point Id End: 128 
 np.array([ 217.96841431  173.6211853    13.25561714]), np.array([ 218.35649109  173.53259277   13.07821465]) 

For cell: 1
Number Of Points: 117 
 Point Id Start: 129 
 Point Id End: 245 
 np.array([ 220.60513306  168.85765076   14.68985176]), np.array([ 220.71549988  168.86035156   14.66243362]) 

For cell: 2
Number Of Points: 123 
 Point Id Start: 246 
 Point Id End: 368 
 np.array([ 220.60513306  161.35403442   14.91808796]), np.array([ 221.12702942  161.36172485   14.86949444]) 

For cell: 3
Number Of Points: 122 
 Point Id Start: 369 
 Point Id End: 490 
 np.array([ 217.96841431  153.79956055   16.24483871]), np.array([ 218.79777527  153.7590332    15.95011616]) 

For cell: 4
Number Of Points: 119 
 Point Id Start: 491 
 Point Id End: 609 
 np.array([ 220.60513306  147.16163635   16.45752907]), np.array([ 220.84371948  147.17149353   16.44560242]) 

For cell: 5
Number Of Points: 102 
 Point Id Start: 610 


In [17]:
print(branches)

vtkPolyData (0x7fb3126b2380)
  Debug: Off
  Modified Time: 1491
  Reference Count: 1
  Registered Events: (none)
  Information: 0x7fb3126b24f0
  Data Released: False
  Global Release Data: Off
  UpdateTime: 51856
  Field Data:
    Debug: Off
    Modified Time: 1387
    Reference Count: 1
    Registered Events: (none)
    Number Of Arrays: 0
    Number Of Components: 0
    Number Of Tuples: 0
  Number Of Points: 2065
  Number Of Cells: 24
  Cell Data:
    Debug: Off
    Modified Time: 1491
    Reference Count: 1
    Registered Events: 
      Registered Observers:
        vtkObserver (0x7fb3126b2a40)
          Event: 33
          EventName: ModifiedEvent
          Command: 0x7fb3126b2770
          Priority: 0
          Tag: 1
    Number Of Arrays: 7
    Array 0 name = BranchSectionGroupIds
    Array 1 name = BranchSectionArea
    Array 2 name = BranchSectionMinSize
    Array 3 name = BranchSectionMaxSize
    Array 4 name = BranchSectionShape
    Array 5 name = BranchSectionClosed
    Arr

In [ ]:
sections.OutputSectionArea

In [ ]:
surfaceviewer = vmtkscripts.vmtkSurfaceViewer()
surfaceviewer.Surface = sections.Surface
surfaceviewer.ArrayName = 'BifurcationSectionGroupIds'
surfaceviewer.Execute()